#### imports

In [12]:
import csv
import requests
import math
import pprint
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime, date, time
from IPython.display import clear_output

In [13]:
cwd = os.getcwd()
print cwd

/Users/jpy/Dropbox/_FLASK/__JUPYTER__/OCF


In [14]:
pp = pprint.PrettyPrinter(indent=2)

#### common functions

In [4]:
def findFromPathString(path_string, JSON, separator="/"):     
  paths = path_string.split(separator)
  data = JSON
  for i in range(0,len(paths)):
    data = data[paths[i]]
  return data

In [5]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

#### settings

In [48]:

root_url_corpos = "http://opencorporatefacts.fr/api/corporates"
# params_corpos = { 'page' : 1 , 'id' : None , 'CompanyNumber' : None }
results_path_corpos = "hydra:member"
output_filename_corpos = "datasets/corporations/corpos"

root_url_accounts = "http://opencorporatefacts.fr/api/compte_de_resultats"
# params_accounts = { 'page' : 1, 'Corporate' : None }
results_path_accounts = "hydra:member"
output_filename_accounts = "datasets/account_results/accounts"

root_url_companies = "http://api.cquest.org/company/" ### pass CompanyNumber after root_url
output_filename_companies = "datasets/companies_data/companies"


#### tests

In [49]:
#print pp.pprint(results)

#### main requests functions

In [ ]:
def generate_filename( output_filename, page_treshold, splite_files_treshold_pages, max_page, only_filename=False ) :
    
    start_page = page_treshold
    end_page = page_treshold + splite_files_treshold_pages - 1
    if end_page > max_page : 
        end_page = max_page
    
    if only_filename : 
        prefix = ''
    else : 
        prefix = cwd + '/'
        
    filename = prefix + output_filename + '-pageStart_' + str(start_page) + '_to_' + str(end_page)
    
    return filename
    
def get_req_results ( 
    url, 
    pages_range, 
    number_of_pages, 
    results_path, 
    output_filename, 
    start_page=1, 
    splite_files_treshold_pages=1000,
    debug=False, 
    out_formats=["csv"] 
    ) : 
    ''' 
    retrieve results from requests 
    and write csv 
    works with API platform only for now
    '''
    
    start_time = datetime.now()
    
    ### count pages
    count = 0
    page_treshold = 0
    
    ### requests and writing file
    
    for page in pages_range : 
    
        if page > 0 and page >= start_page  : 
            
            ### treshold and split into several files
            if page % splite_files_treshold_pages == 0 :
                page_treshold = page
                count = 0
                
            ### output files
            if "csv" in out_formats :
                file_data_csv_filename  = generate_filename( output_filename, page_treshold, splite_files_treshold_pages, number_of_pages )
                file_data_csv  = file_data_csv_filename + '.csv'

            if "json" in out_formats : 
                file_data_json_filename = generate_filename( output_filename, page_treshold, splite_files_treshold_pages, number_of_pages )
                file_data_json = file_data_json_filename + '.json'
            
            clear_output( wait=True ) 
            
            resp = requests.get( url, params = { 'page' : page } ).json()
            results = findFromPathString( results_path, resp )
            
            page_time = datetime.now()
            delta_time = page_time - start_time
            
            filename = generate_filename( output_filename, page_treshold, splite_files_treshold_pages, number_of_pages, only_filename=True )
            
            print "\nPage number : {} / {} \
                    \ndelta_time (after request) : {}\
                    \npage_treshold : {}\
                    \nfilename : {}\
                    \n".format(page, number_of_pages, delta_time, page_treshold, filename)

            # loop results
            for data in results : 
                
                if debug : 
                    # print pp.pprint(data) 
                    print data
                    print
                data_flat = flatten_json(data)
                # print data_flat
                
                ### CSV
                if "csv" in out_formats : 
                    
                    ### creates headers
                    if count == 0:
                        with open(file_data_csv, 'w') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            header = data.keys()
                            csv_writer.writerow(header)
                            count += 1
                    else : 
                        with open(file_data_csv, 'a') as csv_file :
                            csv_writer = csv.writer(csv_file, delimiter=';')
                            csv_writer.writerow( data.values() )
                            count += 1

                        
                ### JSON
                if "json" in out_formats : 
                    if count == 0 :
                        with open(file_data_json, 'w') as json_file :
                            json_file.write(json.dumps( [data] ).encode())         #If empty, write an array
                            count += 1
                    else : 
                        with open(file_data_json, 'ab+') as json_file :
                            json_file.seek(-1,2)           
                            json_file.truncate()                           #Remove the last character, open the array
                            json_file.write(' , '.encode())                #Write the separator
                            json_file.write(json.dumps(data).encode())     #Dump the dictionary
                            json_file.write(']'.encode())                  #Close the array
                            count += 1


#### getting data

In [61]:
### TEST / get accounts 
number_of_pages_accounts_test = 30
pages_range_test = range(number_of_pages_accounts_test)
get_req_results( 
    root_url_accounts, 
    pages_range_test, 
    number_of_pages_accounts_test,
    results_path_accounts, 
    output_filename_accounts+'_test', 
    start_page=1, 
    splite_files_treshold_pages=10,
    debug=True, 
    out_formats=["json"] 
) 


Page number : 29 / 30                     
delta_time (after request) : 0:00:12.225779                    
page_treshold : 20                    
filename : datasets/account_results/accounts_test-pageStart_20                    

{u'DotationProvisions': 0, u'EffectifsMoyens': -1, u'DifferenceNegativeChange': 0, u'RepriseDepreciationEtProvisionTransfertsCharges': 0, u'ResultatFinancier': 0, u'DotationsFinancieresAmortissementDepreciationProvision': 0, u'ChiffresAffairesNet': 51831, u'VariationStockMarchandise': 0, u'year': 2017, u'DotationExceptionnelleAmortissementDepreciationProvision': 0, u'RepriseDepreciationProvisionTransfertCharge': 0, u'AchatsDeMarchandises': 0, u'Corporate': u'/api/corporates/2929', u'id': 841, u'VenteMarchandises': 0, u'ImpotTaxesEtVersementsAssimiles': 1057, u'AutresAchatEtChargesExternes': 17035, u'Benefice': 6617, u'ResultatExploitation': 7785, u'AutreInteretEtProduitAssimile': 0, u'ParticipationSalariesAuxResultats': 0, u'InteretEtChargeAssimilees': 0, u'P

In [52]:
### get corpos data
number_of_results_corpos = 577297
results_per_page = 30
number_of_pages_corpos = int( math.ceil( number_of_results_corpos / results_per_page ) )
print number_of_pages_corpos
pages_to_scrap_corpos = range( number_of_pages_corpos )
print pages_to_scrap_corpos[:3]

start_page_corpos = 1
file_suffix_corpos = '_04'

get_req_results( 
    root_url_corpos, 
    pages_to_scrap_corpos, 
    number_of_pages_corpos,
    results_path_corpos, 
    output_filename_corpos+file_suffix_corpos, 
    start_page=start_page_corpos, 
    splite_files_treshold_pages=1000,
    debug=False, 
    out_formats=["json"] 
) 


Page number : 99 / 19243 
delta_time (after request) : 0:00:42.792007



In [ ]:
### get accounts data
number_of_results_account = 316182
results_per_page = 30
number_of_pages_accounts = int( math.ceil( number_of_results_account / results_per_page ) )
print number_of_pages_accounts
pages_to_scrap_accounts = range( number_of_pages_accounts )
print pages_to_scrap_accounts[:3]

start_page_accounts = 1
file_suffix_accounts = '_05'

get_req_results( 
    root_url_accounts, 
    pages_to_scrap_accounts, 
    number_of_pages_accounts,
    results_path_accounts, 
    output_filename_accounts+file_suffix_accounts, 
    start_page=start_page_accounts, 
    splite_files_treshold_pages=500,
    debug=False, 
    out_formats=["json"] 
) 


Page number : 5825 / 10539                     
delta_time (after request) : 1:12:34.335687                    
page_treshold : 5000                    
filename : datasets/account_results/accounts_04-pageStart_5000                    

